<a href="https://colab.research.google.com/github/albertoliuzzo/progetti_AI/blob/main/Machine_learning_classificazione_frutti_esotici.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

df=pd.read_csv("fruits.csv") #mostro i primi 5 valori del df

print(df.shape)
print("----------------------------")
print(df.count()) #controllo che non ci siano valori mancanti
print("----------------------------")
print(df["Frutto"].value_counts()) #controllo che le classi siano bilanciate

df["Frutto"]=df["Frutto"].map({"Mela":1, "Banana":2, "Arancia":3, "Uva":4, "Kiwi":5}) #conversione del target (Frutto) in variabile numerica

corr_matrix = df.corr() #calcolo e stampo la matrice di correlazione per capire la correlazione tra le variabili stesse e le variabili ed il target
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Matrice di Correlazione tra le Feature")
plt.show()

#creo e stampo uno scatter plot colorando di colori diversi le varie classi
frutti = ["Mela", "Banana", "Arancia", "Uva", "Kiwi"]
colori = ["red", "blue", "orange", "purple", "green"]

for i, frutto in enumerate(frutti):
    idx = np.where(y == i+1)
    plt.scatter(df["Dolcezza (1-10)"].iloc[idx], df["Lunghezza media (mm)"].iloc[idx], color=colori[i], label=frutto)

plt.xlabel("Dolcezza (1-10)")
plt.ylabel("Lunghezza media (mm)")
plt.legend()
plt.show()

X=df.drop(["Frutto"], axis=1) #creo set di training e test
y=df["Frutto"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

mms = MinMaxScaler() #normalizzo i dati con MinMaxScaler
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

from sklearn.model_selection import GridSearchCV #ricerco il miglior valore di k facendo una validazione incrociata a 5 fold
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.linspace(1, 100, 100, dtype=int)}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"Miglior valore di K: {grid_search.best_params_['n_neighbors']}")

best_knn = grid_search.best_estimator_ #restituisce il miglior modello (già fittato) trovato con GridSearch
y_pred_train = best_knn.predict(X_train) #faccio le predizioni per set di training e test e calcolo le probabilità
y_proba_train = best_knn.predict_proba(X_train)
y_pred_test = best_knn.predict(X_test)
y_proba_test = best_knn.predict_proba(X_test)
accuracy_train = accuracy_score(y_train, y_pred_train) #calcolo accuracy e log loss sui set di training e test e li metto a confronto
accuracy_test = accuracy_score(y_test, y_pred_test)
log_train = log_loss(y_train, y_proba_train)
log_test = log_loss(y_test, y_proba_test)
print(f"ACCURACY: TRAIN={accuracy_train:.3f} TEST={accuracy_test:.3f}")
print(f"LOG LOSS: TRAIN={log_train :.3f} TEST={log_test :.3f}")

print("Classification Report (test set):") #creo il classification report per il test di set
print(classification_report(y_test, y_pred_test, target_names=["Mela", "Banana", "Arancia", "Uva", "Kiwi"]))

cm = confusion_matrix(y_test, y_pred_test) #creo e stampo la matrice di confuzione per sapere quali e quanti errori ha commesso il modello
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Mela", "Banana", "Arancia", "Uva", "Kiwi"])
disp.plot(cmap='Blues')
plt.title("Matrice di Confusione (Test set)")
plt.show()